In [1]:
import sys
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import os
import numpy as np
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten,Merge
from keras.models import Model
from keras import backend as K
K.set_image_dim_ordering('tf')
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated2/train_label_.txt", "r") as ins:
    label = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        label.append(int(line))

In [3]:
MAX_SEQUENCE_LENGTH = 98
GLOVE_DIR = ""
EMBEDDING_DIM = 200
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated2/train_s_.txt", "r") as ins:
    texts = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        texts.append(line)

In [4]:
PN_label = []
PN_text = []
Count = 0 

for i in range(0,len(label)/4):
    if (label[i])==2 or (label[i])==3:
        Count = Count + 1
        PN_label.append(label[i])
        PN_text.append(texts[i])
    if Count == 100000:
        break;

In [5]:

#maxi = 0
#for i in range(0,len(PN_label)):
#      if(maxi<len(PN_text[i])): 
#         maxi =  len(PN_text[i])
#print maxi

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(PN_text)
sequences = tokenizer.texts_to_sequences(PN_text)

In [7]:
maxi = 0
for i in range(0,len(PN_label)):
      if(maxi<len(sequences[i])): 
         maxi =  len(sequences[i])
print maxi

98


In [8]:
label = np.asarray(PN_label)  
label[:] = [x - 2 for x in label] 
label = to_categorical(label)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [10]:
GLOVE_DIR = "/home/shyam/Downloads/MS/Sem1/NLP/Project/glove/glove.6B/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
word_index = tokenizer.word_index
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
count1 = 0
count2 = 0
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        count1 = count1 +1
    else:
        count2 = count2 +1

In [12]:
print count1,count2

14622 1267


In [13]:
embedding_layer = Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [14]:
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

lr = 0.0001
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr, decay = lr*0.9),
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()
#model.fit(data,label,validation_split=0.01,nb_epoch=7, batch_size=128)

/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=3, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=4, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=5, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


model fitting - simplified convolutional neural network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 98)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 98, 200)       3178000     input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 96, 128)       76928       embedding_1[0][0]                
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 95, 128)       102528      embedding_1[0][0]                
___________________________________

In [15]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [37]:
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated2/test_label_.txt", "r") as ins:
    label = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        label.append(int(line))
        
MAX_SEQUENCE_LENGTH = 98
GLOVE_DIR = ""
EMBEDDING_DIM = 200
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated2/test_s_.txt", "r") as ins:
    texts = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        texts.append(line)
PN_label = []
PN_text = []
Count = 0 

for i in range(0,len(label)):
    if (label[i])==2 or (label[i])==3:
        Count = Count + 1
        PN_label.append(label[i])
        PN_text.append(texts[i])
sequences = tokenizer.texts_to_sequences(PN_text)
label = np.asarray(PN_label)  
label[:] = [x - 2 for x in label] 
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [38]:
Prediction =  np.argmax(model.predict([data]), axis = 1)

In [39]:
print sum(label==Prediction)

5069


In [40]:
5103.0/6605

0.7725965177895534